# Recalcular indicadores por buffer y para niveles de agregación superiores

In [1]:
import shapefile
from scripts import utils
from scripts import recalculate_indicators
import pandas as pd
from scripts.path_finders import get_weights_path, get_indicators_path, get_indicators_shp_path
from scripts.create_indicators import get_or_create_indicators_df
from scripts.geo_utils import join_df_with_shp
from scripts.path_finders import get_division_path
from scripts.data_loaders import read_dbf

PyTables is not installed. No support for HDF output.


## 1. Recalcular indicadores de radios censales para cada buffer

Los indicadores calculados por buffer son un promedio ponderado de los indicadores de radios censales que intersectan el buffer. Recalculamos los indicadores para todos los shapefiles generados con buffers de puntos de transporte.

In [2]:
# se omiten aquellos indicadores que son cuentas, y por lo tanto su agregación no es una media
omit_fields = ["hospitales", "educ_priv", "educ_pub"]
by_area = ["COMERCIAL", "INDUSTRIAL", "SERVICIOS", "OTROS", "RESIDENCIAL"]
recalculate_indicators.main(recalculate=True, area_level="RADIO", 
                            subcategory="buffers", omit_fields=omit_fields, by_area=by_area)

Calculating estaciones_de_ferrocarril-buffer1000 indicators
Calculating estaciones_de_ferrocarril-buffer1500 indicators
Calculating estaciones_de_ferrocarril-buffer2000 indicators
Calculating estaciones_de_ferrocarril-buffer300 indicators
Calculating estaciones_de_ferrocarril-buffer500 indicators
Calculating estaciones_de_ferrocarril-buffer750 indicators
Calculating estaciones_de_metrobus-buffer1000 indicators
Calculating estaciones_de_metrobus-buffer1500 indicators
Calculating estaciones_de_metrobus-buffer2000 indicators
Calculating estaciones_de_metrobus-buffer300 indicators
Calculating estaciones_de_metrobus-buffer500 indicators
Calculating estaciones_de_metrobus-buffer750 indicators
Calculating estaciones_de_premetro-buffer1000 indicators
Calculating estaciones_de_premetro-buffer1500 indicators
Calculating estaciones_de_premetro-buffer2000 indicators
Calculating estaciones_de_premetro-buffer300 indicators
Calculating estaciones_de_premetro-buffer500 indicators
Calculating estacione

## 2. Calcular indicadores para niveles de agregación superiores

Algunos indicadores fueron calculados sólo para un nivel de agregación, pero podrían agregarse para los niveles subsiguientes (los indicadores calculados por radio censal, pueden agregarse por fracción, comuna o barrio). Antes de crear los shps finales, se recalcularán aquellos indicadores que sólo estén presentes en un nivel de agregación pero pueden estar en otros.

In [18]:
indicators_radio = get_or_create_indicators_df("RADIO")
indicators_frac = get_or_create_indicators_df("FRAC")
indicators_comuna = get_or_create_indicators_df("DPTO")
indicators_caba = get_or_create_indicators_df("PROV")

In [19]:
indicators_comuna = indicators_comuna[indicators_comuna.index != "COMERCIAL"]

In [20]:
def get_id_frac(id_radio):
    comuna, frac, ra = id_radio.split("_")
    return "_".join([comuna.zfill(3), frac])

def get_id_comuna(id_radio):
    return unicode(id_radio.split("_")[0])

indicators_radio["CO_FRACC"] = map(get_id_frac, indicators_radio["CO_FRAC_RA"])
indicators_radio["COMUNAS"] = map(get_id_comuna, indicators_radio["CO_FRAC_RA"])
indicators_radio["PROV"] = "02"

In [21]:
def to_string(x):
    if type(x) == str or type(x) == unicode:
        return x
    else:
        return str(int(x))

In [22]:
def reaggregate_indicators(df_from, df_to, id_area_level, omit_fields, area_indics=None,
                           recalc_indics=None, pop_field="hab", area_field="AREA"):
    area_indics = area_indics or []
    recalc_indics = recalc_indics or []
    
    # make a copy, so intermediate data is not stored in the data frame
    dfr = df_from.copy()
    
    hab_to = dfr.groupby(by=id_area_level).sum()[pop_field]
    dfr["hab_to"] = dfr[id_area_level].map(hab_to.to_dict())
    dfr["pond_hab_to"] = dfr[pop_field] / dfr["hab_to"]

    area_to = dfr.groupby(by=id_area_level).sum()[area_field]
    dfr["area_to"] = dfr[id_area_level].map(area_to.to_dict())
    dfr["pond_area_to"] = dfr[area_field] / dfr["area_to"]
    
    
    for indic in df_from.columns:
        if indic not in omit_fields and (indic not in df_to.columns or 
                                         pd.isnull(df_to[indic]).all() or
                                         indic in recalc_indics):

            if indic in area_indics:
                dfr["temp"] = dfr["pond_area_to"] * dfr[indic]
            else:
                dfr["temp"] = dfr["pond_hab_to"] * dfr[indic]
            indic_dict = dfr.groupby(by=id_area_level).sum()["temp"].to_dict()

            if id_area_level in df_to.columns:
                # ensure ids are strings
                df_to.loc[:,id_area_level] = df_to.loc[:,id_area_level].apply(lambda x: to_string(x))
                df_to[indic] = df_to[id_area_level].map(indic_dict) 

            else:
                df_to[indic] = dfr.groupby(by=id_area_level).sum()["temp"]
            #print df_to[[indic, id_area_level]]

omit_fields = ["educ_priv", "educ_pub", "hospitales", "CO_FRAC_RA", "COMUNAS", "CO_FRACC", "PROV", "AREA", "area_km2"]
area_indics = ["esp_verde", "RESIDENCIAL", "COMERCIAL", "INDUSTRIAL", "SERVICIOS", "OTROS"]
recalc_indics = ["esp_verde", "RESIDENCIAL", "COMERCIAL", "INDUSTRIAL", "SERVICIOS", "OTROS", 
                 "reach_prop", "reach_area", "nse_mex"]

reaggregate_indicators(indicators_radio, indicators_frac, "CO_FRACC", omit_fields, area_indics, recalc_indics)
reaggregate_indicators(indicators_radio, indicators_comuna, "COMUNAS", omit_fields, area_indics, recalc_indics)
reaggregate_indicators(indicators_radio, indicators_caba, "PROV", omit_fields, area_indics, recalc_indics)

In [23]:
indicators_frac.to_csv(get_indicators_path("FRAC"), encoding="utf-8")
indicators_comuna.to_csv(get_indicators_path("DPTO"), encoding="utf-8")
indicators_caba.to_csv(get_indicators_path("PROV"), encoding="utf-8")

## 3. Crear shapefiles de los indicadores

In [24]:
join_df_with_shp(get_division_path("radios_censo_2010"), indicators_radio.set_index("CO_FRAC_RA"), "indicadores")
join_df_with_shp(get_division_path("fracciones_caba_censo_2010"), indicators_frac.set_index("CO_FRACC"), "indicadores")
join_df_with_shp(get_division_path("comunas_caba_censo_2010"), indicators_comuna.set_index("COMUNAS"), "indicadores")

## 4. Recalcular indicadores por barrio

Los barrios son unidades geográficas no construidas como conjuntos de radios o fracciones censales, de manera que el recálculo de indicadores para este nivel de agregación no se puede hacer con joins basados en un sistema de ids. Para este caso, utilizamos un procedimiento similar al del recálculo de indicadores por buffer.

In [25]:
from scripts.data_loaders import get_weights

In [26]:
barrios_shp_dir = "shp/divisiones/barrios_censo_2010"

In [27]:
recalculate_indicators.recalculate_indicators(barrios_shp_dir, area_level="RADIO", 
                                              omit_fields=omit_fields, by_area=by_area)

Chequeamos que la cantidad de población calculada por barrio sea consistente con el total de la CABA.

In [28]:
shp_indic_barrios = "indicadores/barrios_censo_2010/barrios_censo_2010"
shp_indic_comunas = "indicadores/comunas_censo_2010/comunas_censo_2010"
shp_indic_fracciones = "indicadores/fracciones_censo_2010/fracciones_censo_2010"
shp_indic_radios = "indicadores/radios_censo_2010/radios_censo_2010"

In [29]:
def total_caba_population(shp_indicators):
    sf = shapefile.Reader(shp_indicators)
    pop_index = [field[0] for field in sf.fields[1:]].index("hab")
    return sum((record[pop_index] for record in sf.iterRecords() if record[pop_index] is not None))

In [30]:
print total_caba_population(shp_indic_barrios)
print total_caba_population(shp_indic_comunas)
print total_caba_population(shp_indic_fracciones)
print total_caba_population(shp_indic_radios)

print "Margen de error", total_caba_population(shp_indic_barrios) / total_caba_population(shp_indic_comunas) - 1

2889007.41538
2890151.0
2890151.0
2890151.0
Margen de error -0.000395683345081


## 5. Chequear qué indicadores tienen missings en cada nivel de agregación

In [31]:
import json

In [32]:
with open("prueba.json", "wb") as f:
    json.dump(json.dumps({"hola": "prueba"}), f) 